# Set up, data, dataframe creation

In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

#Set working directory 
#os.chdir('/Users/Davide/Documents/University/RA/text_analysis')
os.getcwd()


'/Users/Davide/Documents/University/RA/text_analysis/sentiment_analysis-master'

In [52]:
#scraped data
headlines_df = pd.read_csv('express.csv', index_col = 'date')

daily_mail_df = pd.read_excel('daily_mail_after_filters.xlsx', index_col = 'date')
evening_standard_df = pd.read_excel('evening_standard_after_filters.xlsx', index_col = 'date')
express_df = pd.read_excel('express_after_filters.xlsx', index_col = 'date')
guardian_df = pd.read_excel('guardian_after_filters.xlsx', index_col = 'date')
independent_df = pd.read_excel('independent_after_filters.xlsx', index_col = 'date')
times_df = pd.read_excel('times_after_filters.xlsx', index_col = 'date')

#Concatenating all articles together
newspapers = [daily_mail_df, evening_standard_df, express_df, guardian_df, independent_df, times_df]
articles_df = pd.concat(newspapers)

#labeled data
bbc_df = pd.read_csv('News_dataset.csv', sep=';')
kaggle_df = pd.read_json (r'/Users/Davide/Documents/University/RA/text_analysis/sentiment_analysis-master/News_Category_Dataset_v2.json', lines = True)

In [3]:
articles_df

,headline,newspaper,author,page,text
date,,,,,
1-3-2015,Two-thirds of laws in Britain over the past tw...,Daily Mail,"Tamara Cohen, Political Correspondent for the ...",NaN,Study found 64.7 per cent of the laws made bet...
1-3-2015,Now Europe wants to ban your halogen light bul...,Daily Mail,"Sean Poulter, Consumer Affairs Editor For The ...",NaN,Follows ban of incandescent bulbs in bid to cu...
1-3-2015,Scott Walker admits flip-flop on illegal immig...,Daily Mail,Associated Press,NaN,Wisconsin governor once favored plan whereby u...
1-3-2015,"Don't blame immigrants for ills of society, sa...",Daily Mail,"Matt Chorley, Political Editor for MailOnline",NaN,"Letter to 500,000 parishes warns of blaming im..."
2-3-2015,Landmark EU ruling to cut plastic bag use by 8...,Daily Mail,Sean Poulter for the Daily Mail,NaN,EU decision is a victory for Daily Mail reader...
...,...,...,...,...,...
1-3-2017,Warning that future depends on better infrastr...,The Times,Peter O’Dwyer,NaN,Ireland’s leading construction lobby group h...
24-3-2017,Your five-minute digest,The Times,NaN,NaN,1 Gambling bosses are at war over fixed-odds...
27-3-2017,Your five-minute digest,The Times,NaN,NaN,TodayThe Bank of England’s financial policy ...


In [4]:
headlines_df

,headline,newspaper
date,,
1-3-2015,Prince Andrew 'frozen out by Charles over dama...,Express
1-3-2015,EXCLUSIVE: Migrants to put Britain's populatio...,Express
1-3-2015,EXCLUSIVE: Jihadi John exposed by web error: K...,Express
1-3-2015,Missing Becky Watts: Two arrested as family ad...,Express
1-3-2015,We can all benefit from a positive approach to...,Express
...,...,...
31-3-2017,Man United player: I was worried after manager...,Express
31-3-2017,Mark Lawrenson: My biggest worry about Liverpool,Express
31-3-2017,La Liga ace reveals messages received from Man...,Express


In [5]:
bbc_df

,File_Name,Content,Category,Complete_Filename
0,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,business,001.txt-business
1,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,business,002.txt-business
2,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,business,003.txt-business
3,004.txt,High fuel prices hit BA's profits\n\nBritish A...,business,004.txt-business
4,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,business,005.txt-business
...,...,...,...,...
2220,397.txt,BT program to beat dialler scams\n\nBT is intr...,tech,397.txt-tech
2221,398.txt,Spam e-mails tempt net shoppers\n\nComputer us...,tech,398.txt-tech
2222,399.txt,Be careful how you code\n\nA new European dire...,tech,399.txt-tech
2223,400.txt,US cyber security chief resigns\n\nThe man mak...,tech,400.txt-tech


In [6]:
#Slice the df to have only Green and environment articles
green_df = kaggle_df[kaggle_df.category == 'GREEN']
env_df = kaggle_df[kaggle_df.category == 'ENVIRONMENT']


# Extracting Keywords

In [7]:
# Cleaning
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
nltk.download('wordnet')


def cleaning(df, column_str):
    '''function to clean a text within a column of a dataframe. The column name has to be a string'''
    #Special character
    df[column_str] = df[column_str].str.replace("\r", " ")
    df[column_str] = df[column_str].str.replace("\n", " ")
    df[column_str] = df[column_str].str.replace("    ", " ")
    df[column_str] = df[column_str].str.replace('"', '')

    #Lowercase
    df[column_str] = df[column_str].str.lower()

    #Punctuation
    punctuation_signs = list("?:!.,;-")

    for punct_sign in punctuation_signs:
        df[column_str] = df[column_str].str.replace(punct_sign, '')

    #Possessive pronouns
    df[column_str] = df[column_str].str.replace("'s", "")
    df[column_str] = df[column_str].str.replace("'", "")

    #Lemmatization
    # Saving the lemmatizer into an object
    wordnet_lemmatizer = WordNetLemmatizer()

    nrows = len(df)
    lemmatized_text_list = []

    for row in range(0, nrows):

        # Create an empty list containing lemmatized words
        lemmatized_list = []

        # Save the text and its words into an object
        text = df.iloc[row][column_str]
        text_words = text.split(" ")

        # Iterate through every word to lemmatize
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

        # Join the list
        lemmatized_text = " ".join(lemmatized_list)

        # Append to the list containing the texts
        lemmatized_text_list.append(lemmatized_text)

    df[column_str] = lemmatized_text_list

    #Stopwords
    # Downloading the stop words list
    nltk.download('stopwords')
    # Loading the stop words in english
    stop_words = list(stopwords.words('english'))

    df[column_str] = df[column_str]

    for stop_word in stop_words:

        regex_stopword = r"\b" + stop_word + r"\b"
        df[column_str] = df[column_str].str.replace(regex_stopword, '')

    return df

[nltk_data] Downloading package punkt to /Users/Davide/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Davide/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
green_df = cleaning(green_df, 'headline')
bbc_df = cleaning(bbc_df, 'Content')
env_df = cleaning(env_df, 'headline')

/Users/Davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/Davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/Davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [9]:
biz_df = bbc_df[bbc_df.Category == 'business']
pol_df = bbc_df[bbc_df.Category == 'politics']

In [10]:
#Most commonly used words extractor
def keywords_extractor(df, headline_str, n):
    '''Return the most commonly used words from a dataframe with a column containing text.
    Arguments are dataframe, the name of the column in string format, and the number n of keywords needed'''
    df_words = Counter()
    df[headline_str].str.split().apply(df_words.update)
    df_words = df_words.most_common(20)
    df_words_lst = []
    for tup in df_words:
        df_words_lst.append(tup[0])
    return df_words_lst

#Keywords list creation
biz_keywords = keywords_extractor(biz_df, 'Content', n = 20)
pol_keywords = keywords_extractor(pol_df, 'Content', n = 20)
green_keywords = keywords_extractor(green_df, 'headline', n = 20)
env_keywords = keywords_extractor(env_df, 'headline', n = 20)

In [11]:
print(biz_keywords)

['say', 'us', 'year', 'company', 'mr', 'firm', 'market', 'would', 'bank', 'rise', 'also', 'new', 'price', 'share', 'growth', 'last', 'economy', 'make', 'government', 'sales']


In [12]:
print(pol_keywords)

['say', 'mr', 'would', 'labour', 'government', 'party', 'people', 'blair', 'election', 'minister', 'plan', 'make', 'also', 'new', 'tell', 'could', 'brown', 'go', 'tax', 'lord']


In [13]:
#Combining green and env keywords
climate_keywords = []
climate_keywords.extend(green_keywords)
climate_keywords.extend(env_keywords)

#Eliminating duplicates
climate_keywords = list(set(climate_keywords))

#Removing not useful keywords
climate_keywords.remove('(photos)')
climate_keywords.remove('(video)')

#Including some additional important words
climate_keywords.extend(["climate change", "paris agreement", "environment", "global warming", "unfccc"])

print(climate_keywords)

['climate', 'kill', 'california', 'day', 'dog', 'oil', 'week', 'find', 'extreme', 'energy', 'get', 'make', 'save', 'us', 'green', 'change', 'trump', 'animal', 'picture', 'world', 'water', 'could', 'take', 'say', 'photos', 'watch', 'weather', '2012', 'baby', 'hurricane', 'new', 'climate change', 'paris agreement', 'environment', 'global warming', 'unfccc']


In [14]:
british_keywords = ["brexit", "european union", " eu ", "british", "british identity", "british passport",
                    "british culture", "british heritage", "british goods", "british products","british manufacturing",
                    "made in britain"]
immigration_keywords = ['migrant', 'refugee', 'immigrant', 'asylum', 'Calais']


# Classification

In [53]:
#Defining the functions that need to be used 


def topic_classifier(string, keywords_tuple, name_class_tuple):
    '''Classifies a list of strings according to keywords lists presented inside a tuple.
    Also requires the names of the classes in string from in a second tuple'''
    string = string.lower()
    if len(keywords_tuple) == len(name_class_tuple) and len(keywords_tuple) > 0:
        no_of_classes = len(keywords_tuple)
        counts = []
        count = 0
        while no_of_classes > 0:
            no_of_classes -= 1
            for keyword in keywords_tuple[no_of_classes]:
                if keyword in string:
                    count += 1
            counts.insert(0, count)
            count = 0
    else:
        return "List of keywords do not match list of classes or the list of keywords is empty"
        
    if max(counts) > 0:
        max_list = max(counts)
        values = np.array(counts)
        ii = np.where(values==max_list)[0]
        results_temp = [name_class_tuple[index] for index in ii]
        sep = ","
        results = sep.join(results_temp)
        return results
    else:
        return 'Other'


# def headline_classifier(string, keyword_lst_class_1, keyword_lst_class_2, name_class_1 = 'class_1', name_class_2 = 'class_2'):
#     '''Classifies a list of strings according to keyword lists for class_1 and class_2'''
#     string = string.lower()
#     count_1 = 0
#     count_2 = 0
#     for keyword in keyword_lst_class_1:
#         if keyword in string:
#             count_1 =+ 1
#     for keyword in keyword_lst_class_2:

#         if keyword in string:
#             count_2 =+ 1
    
#     if count_1 > 0 and count_2 > 0:
#         return 'both'
#     elif count_1 > 0:
#         return name_class_1
#     elif count_2 > 0:
#         return name_class_2
#     else:
#         return 'none'
    
def headline_classification(data, keywords_tuple, name_class_tuple):
    n_row, n_col = data.shape
    data_lst = list(data['headline'].values)

    #Headline classification
    classification_lst = []
    for i in range(n_row):
        data_i = str(data_lst[i]) #Include this becasue of potential nan values
        classification_lst.append(topic_classifier(data_i, keywords_tuple, name_class_tuple))
    return classification_lst

def text_classification(data, keywords_tuple, name_class_tuple):
    n_row, n_col = data.shape
    data_lst = list(data['text'].values)

    #Text classification
    classification_lst = []
    for i in range(n_row):
        data_i = str(data_lst[i]) #Include this becasue of potential nan values
        classification_lst.append(topic_classifier(data_i, keywords_tuple, name_class_tuple))
    return classification_lst

def sentiment_analysis (df, column_str):
    '''function needs a dataframe with a string column where to perfrom sentiment analysis
    provide the column name in string format'''
    n_row, n_col = df.shape
    headlines_lst = list(df[column_str].values)
    #Empty list to add the polarity score
    polarity_lst = []
    subjectivity_lst = []

    #Headline sentiment
    for i in range(n_row):
        headline_i = str(headlines_lst[i]) #Include this becasue of potential nan values
        blob_headline_i = TextBlob(headline_i) #transforming string into textblob
        polarity_lst.append(blob_headline_i.sentiment.polarity)
        subjectivity_lst.append(blob_headline_i.sentiment.subjectivity)

    #Adding polarity and subjectivity scores to the headlines dataframe
    df['polarity '+ column_str] = polarity_lst
    df['subjectivity '+ column_str] = subjectivity_lst
    return df

In [89]:
#Daily mail first classification
key_tup = (british_keywords, climate_keywords, biz_keywords, pol_keywords, immigration_keywords)
name_tup = ('Brexit', 'Climate', 'Business', 'Politics', 'Immigration')
newspaper_lst = [daily_mail_df, evening_standard_df, express_df, guardian_df, independent_df, times_df, articles_df]

# for i in range(len(newspaper_lst)):
#     df_x = newspaper_lst[i]
#     df_x['classification'] = ''
#     class_lst = headline_classification(df_x, key_tup, name_tup)
#     newspaper_lst[i] = df_x.assign(classification = class_lst)

#headline classification
daily_mail_df['headline_classification'] = ''
head_class_lst = headline_classification(daily_mail_df, key_tup, name_tup)
daily_mail_df = daily_mail_df.assign(headline_classification = head_class_lst)
#sentiment_analysis
sentiment_analysis(daily_mail_df, 'headline')

#text classification
daily_mail_df['text_classification'] = ''
text_class_lst = text_classification(daily_mail_df, key_tup, name_tup)
daily_mail_df = daily_mail_df.assign(text_classification = text_class_lst)
#sentiment analysis
sentiment_analysis(daily_mail_df, 'text')

daily_mail_df.text_classification.value_counts()

Climate                                         3263
Business                                        1613
Politics                                        1598
Climate,Business                                 772
Business,Politics                                520
Climate,Politics                                 487
Climate,Business,Politics                        406
Immigration                                      177
Brexit                                           126
Climate,Immigration                              118
Brexit,Climate,Business                           74
Brexit,Climate,Business,Politics                  72
Brexit,Climate                                    69
Climate,Business,Politics,Immigration             69
Business,Immigration                              63
Climate,Business,Immigration                      61
Brexit,Politics                                   53
Brexit,Business                                   46
Climate,Politics,Immigration                  

In [91]:
#Evening Standard first classification
#headline classification
evening_standard_df['headline_classification'] = ''
head_class_lst = headline_classification(evening_standard_df, key_tup, name_tup)
evening_standard_df = evening_standard_df.assign(headline_classification = head_class_lst)
#sentiment_analysis
sentiment_analysis(evening_standard_df, 'headline')

#text classification
evening_standard_df['text_classification'] = ''
text_class_lst = text_classification(evening_standard_df, key_tup, name_tup)
evening_standard_df = evening_standard_df.assign(text_classification = text_class_lst)
#sentiment analysis
sentiment_analysis(evening_standard_df, 'text')

evening_standard_df.text_classification.value_counts()

Climate                                         743
Politics                                        594
Business                                        390
Climate,Business                                178
Business,Politics                               178
Climate,Business,Politics                       141
Climate,Politics                                127
Brexit                                           43
Brexit,Climate                                   18
Brexit,Climate,Business,Politics                 18
Brexit,Politics                                  16
Brexit,Climate,Politics                          14
Climate,Immigration                              13
Brexit,Business,Politics                         11
Climate,Business,Immigration                     11
Brexit,Climate,Business                          10
Brexit,Business                                  10
Immigration                                       9
Climate,Business,Politics,Immigration             7
Business,Pol

In [92]:
#Express fist classification
#headline classification
express_df['headline_classification'] = ''
head_class_lst = headline_classification(express_df, key_tup, name_tup)
express_df = express_df.assign(headline_classification = head_class_lst)
#sentiment_analysis
sentiment_analysis(express_df, 'headline')

#text classification
express_df['text_classification'] = ''
text_class_lst = text_classification(express_df, key_tup, name_tup)
express_df = express_df.assign(text_classification = text_class_lst)
#sentiment analysis
sentiment_analysis(express_df, 'text')

express_df.text_classification.value_counts()

Politics                                        4430
Climate                                         3230
Business                                        2688
Business,Politics                               1306
Climate,Business                                 893
Climate,Politics                                 696
Climate,Business,Politics                        631
Brexit                                           398
Immigration                                      210
Brexit,Climate,Business,Politics                 184
Brexit,Climate                                   159
Brexit,Politics                                  149
Brexit,Climate,Business                          146
Brexit,Business                                  144
Brexit,Business,Politics                         144
Climate,Immigration                              135
Climate,Business,Immigration                     125
Brexit,Climate,Politics                           95
Climate,Business,Politics,Immigration         

In [93]:
#Guardian first classification
#headline classification
guardian_df['headline_classification'] = ''
head_class_lst = headline_classification(guardian_df, key_tup, name_tup)
guardian_df = guardian_df.assign(headline_classification = head_class_lst)
#sentiment_analysis
sentiment_analysis(guardian_df, 'headline')

#text classification
guardian_df['text_classification'] = ''
text_class_lst = text_classification(guardian_df, key_tup, name_tup)
guardian_df = guardian_df.assign(text_classification = text_class_lst)
#sentiment analysis
sentiment_analysis(guardian_df, 'text')

guardian_df.text_classification.value_counts()

Climate                                  277
Politics                                 194
Business                                 189
Climate,Business                          70
Climate,Politics                          51
Business,Politics                         51
Climate,Business,Politics                 34
Brexit                                    18
Brexit,Climate,Business,Politics          16
Brexit,Climate,Business                   15
Brexit,Climate                            13
Brexit,Climate,Politics                   11
Brexit,Politics                            8
Immigration                                6
Brexit,Business                            5
Climate,Immigration                        5
Business,Immigration                       3
Climate,Business,Politics,Immigration      2
Brexit,Business,Politics                   2
Brexit,Politics,Immigration                1
Brexit,Climate,Business,Immigration        1
Brexit,Immigration                         1
Climate,Bu

In [94]:
#Independent first classification
#headline classification
independent_df['headline_classification'] = ''
head_class_lst = headline_classification(independent_df, key_tup, name_tup)
independent_df = independent_df.assign(headline_classification = head_class_lst)
#sentiment_analysis
sentiment_analysis(independent_df, 'headline')

#text classification
independent_df['text_classification'] = ''
text_class_lst = text_classification(independent_df, key_tup, name_tup)
independent_df = independent_df.assign(text_classification = text_class_lst)
#sentiment analysis
sentiment_analysis(independent_df, 'text')

independent_df.text_classification.value_counts()

Politics                                        2362
Climate                                         2290
Business                                        1726
Business,Politics                                672
Climate,Business                                 551
Climate,Politics                                 400
Climate,Business,Politics                        350
Brexit                                           218
Brexit,Politics                                   99
Brexit,Climate,Business,Politics                  95
Immigration                                       87
Brexit,Climate                                    81
Brexit,Business                                   74
Brexit,Climate,Business                           73
Brexit,Business,Politics                          66
Brexit,Climate,Politics                           61
Climate,Immigration                               57
Climate,Business,Immigration                      37
Business,Immigration                          

In [95]:
#Times first classification
#headline classification
times_df['headline_classification'] = ''
head_class_lst = headline_classification(times_df, key_tup, name_tup)
times_df = times_df.assign(headline_classification = head_class_lst)
#sentiment_analysis
sentiment_analysis(times_df, 'headline')

#text classification
times_df['text_classification'] = ''
text_class_lst = text_classification(times_df, key_tup, name_tup)
times_df = times_df.assign(text_classification = text_class_lst)
#sentiment analysis
sentiment_analysis(times_df, 'text')

times_df.text_classification.value_counts()

Climate                                         3963
Business                                        3838
Politics                                        2844
Climate,Business                                1101
Business,Politics                                904
Climate,Politics                                 661
Climate,Business,Politics                        578
Brexit                                           224
Brexit,Climate,Business                          132
Immigration                                      105
Brexit,Climate                                    99
Brexit,Climate,Business,Politics                  97
Brexit,Business                                   96
Brexit,Politics                                   75
Brexit,Business,Politics                          65
Brexit,Climate,Politics                           49
Climate,Immigration                               47
Climate,Business,Politics,Immigration             41
Climate,Business,Immigration                  

In [96]:
#All articles first classification
#headline classification
articles_df['headline_classification'] = ''
head_class_lst = headline_classification(articles_df, key_tup, name_tup)
articles_df = articles_df.assign(headline_classification = head_class_lst)
#sentiment_analysis
sentiment_analysis(articles_df, 'headline')

#text classification
articles_df['text_classification'] = ''
text_class_lst = text_classification(articles_df, key_tup, name_tup)
articles_df = articles_df.assign(text_classification = text_class_lst)
#sentiment analysis
sentiment_analysis(articles_df, 'text')

articles_df.text_classification.value_counts()

Climate                                         13766
Politics                                        12022
Business                                        10444
Business,Politics                                3631
Climate,Business                                 3565
Climate,Politics                                 2422
Climate,Business,Politics                        2140
Brexit                                           1027
Immigration                                       594
Brexit,Climate,Business,Politics                  482
Brexit,Climate,Business                           450
Brexit,Climate                                    439
Brexit,Politics                                   400
Climate,Immigration                               375
Brexit,Business                                   375
Brexit,Business,Politics                          317
Climate,Business,Immigration                      276
Brexit,Climate,Politics                           259
Climate,Business,Politics,Im

In [17]:
#If we want to classify Brexit terms only with biz/pol/immigration 
#Slice of only the Brexit terms
brexit_df = daily_mail_df[daily_mail_df.classification == 'Brexit']

#Further classification for Brexit articles
key_tup2 = (biz_keywords, pol_keywords, immigration_keywords)
name_tup2 = ('Business', 'Politics', 'Immigration')
classification_lst = []

daily_mail_brexit_class = headline_classification(brexit_df, key_tup2, name_tup2)
brexit_df = brexit_df.assign(classification = daily_mail_brexit_class)
brexit_df.classification.value_counts()

Other                            411
Politics                         374
Business                         230
Business,Politics                111
Immigration                       76
Politics,Immigration              24
Business,Politics,Immigration      8
Business,Immigration               8
Name: classification, dtype: int64

# Sentiment Analysis

In [88]:
def sentiment_analysis (df, column_str):
    '''function needs a dataframe with a string column where to perfrom sentiment analysis
    provide the column name in string format'''
    n_row, n_col = df.shape
    headlines_lst = list(df[column_str].values)
    #Empty list to add the polarity score
    polarity_lst = []
    subjectivity_lst = []

    #Headline sentiment
    for i in range(n_row):
        headline_i = str(headlines_lst[i]) #Include this becasue of potential nan values
        blob_headline_i = TextBlob(headline_i) #transforming string into textblob
        polarity_lst.append(blob_headline_i.sentiment.polarity)
        subjectivity_lst.append(blob_headline_i.sentiment.subjectivity)

    #Adding polarity and subjectivity scores to the headlines dataframe
    df['polarity '+ column_str] = polarity_lst
    df['subjectivity '+ column_str] = subjectivity_lst
    return df

In [27]:
sentiment_analysis(daily_mail_df, 'headline')

,headline,newspaper,author,page,text,classification,polarity,subjectivity
date,,,,,,,,
1-3-2015,Two-thirds of laws in Britain over the past tw...,Daily Mail,"Tamara Cohen, Political Correspondent for the ...",NaN,Study found 64.7 per cent of the laws made bet...,Climate,-0.250000,0.250000
1-3-2015,Now Europe wants to ban your halogen light bul...,Daily Mail,"Sean Poulter, Consumer Affairs Editor For The ...",NaN,Follows ban of incandescent bulbs in bid to cu...,Climate,0.175000,0.275000
1-3-2015,Scott Walker admits flip-flop on illegal immig...,Daily Mail,Associated Press,NaN,Wisconsin governor once favored plan whereby u...,Climate,-0.319444,0.402778
1-3-2015,"Don't blame immigrants for ills of society, sa...",Daily Mail,"Matt Chorley, Political Editor for MailOnline",NaN,"Letter to 500,000 parishes warns of blaming im...",Climate,0.000000,0.100000
2-3-2015,Landmark EU ruling to cut plastic bag use by 8...,Daily Mail,Sean Poulter for the Daily Mail,NaN,EU decision is a victory for Daily Mail reader...,Climate,0.000000,0.000000
...,...,...,...,...,...,...,...,...
31-3-2017,People smuggler who 'drove into the US with fo...,Daily Mail,Mail Online Reporter,NaN,"Jose Emiliano Aguilar, 24, is accused of tryin...",Climate,0.000000,0.500000
31-3-2017,Bank of England keeps the vegans happy with pl...,Daily Mail,Giulia Crouch For Mailonline,NaN,The plastic note was slammed by vegans when it...,Climate,0.485227,0.727273
31-3-2017,Women putting themselves at risk by being pros...,Daily Mail,Rebecca Taylor For Mailonline,NaN,Victoria Bateman called the ban on prostitutio...,Climate,0.200000,0.600000


In [ ]:
#Creating dataframe for each class
brexit_df = headlines_df[headlines_df.classification == 'Brexit']
climate_df = headlines_df[headlines_df.classification == 'Climate']
both_df = headlines_df[headlines_df.classification == 'both']

#
results_df = pd.DataFrame(index = ['polarity', 'subjectivity'], columns = ['Brexit', 'Climate', 'Both'])
results_df['Brexit'] = brexit_df.mean(axis = 0, numeric_only = True)
results_df['Climate'] = climate_df.mean(axis = 0, numeric_only = True)
results_df['Both'] = both_df.mean(axis = 0, numeric_only = True)

results_df

In [ ]:
# n_row, n_col = headlines_df.shape
# headlines_lst = list(headlines_df['headline'].values)

# #Headline classification
# classification_lst = []
# for i in range(n_row):
#     headline_i = str(headlines_lst[i]) #Include this becasue of potential nan values
#     classification_lst.append(headline_classifier(headline_i, british_keywords, climate_keywords, name_class_1 = 'Brexit', name_class_2 = 'Climate'))

# headlines_df['classification'] = classification_lst
# headlines_df.head(50)
# headlines_df.classification.value_counts()

# #Testing
# #To do find more elegant way to deal with eu than " eu "
# test_lst = []

# test_headline = headlines_df.iloc[43]['headline']     
# test_lst.append(headline_classifier(test_headline, british_keywords, climate_keywords, name_class_1 = 'Brexit', name_class_2 = 'Climate'))              
# test_lst

# test_lst
